# A Brief Introduction to Scipy

## Learning Outcomes

By the end of this notebook you will know how to:
- Perform simple fitting with on data using functions.
- Perform simple integration.
- Perform simple interpolation. 


Scipy is a huge package with lots of modules including the `optimize`, `integrate`, and `interpolate` modules we'll use here. Beyond these their are modules for coordinate and geometry calculations (`spatial`), signal processing (`signal`), statistics (`stats`), and many many others. If you're doing something scientific, it almost certainly has efficient and robust implementations you can make use of! 

In this notebook, I'll cover 3 examples which will be important in your degree moving forward.

## Basic Curve Fitting

As you no doubt have already seen, when working with scientific data you're often fitting your experimental data to a model or theory. You can then compute statistics to determine the validity of any theories you may be comparing to. Although you can do this in excel with `LINEST`, this is very limited both in implementation and in the logistics of working with large data sets. Instead we want to utilise `scipy.optimize` which contains a fitting function called `curve_fit` (amongst others). Here I'll demonstrate how to use `curve_fit` so that your ready when you need to use it.

In the data directory you will find `curvefitting.txt`. Below we will load this data into the notebook and then fit it with a straight line. 

First we need to load it. It's a text file so we can use `numpy` for this.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Load the data
arr = np.loadtxt('../data/curvefitting.txt')

# Convert the data from strings to floats
arr = np.float64(arr)

# Extract the xs and ys from the file
xs = arr[:, 0]
ys = arr[:, 1]

We now need a function to fit the data. For that we can just define a straight line.

In [ ]:
def func(x, m, c):
    """
    A straight line to fit the data
    """
    return m * x + c


Finally we need some initial guesses for the parameters (here these are `m` and `c`).

In [ ]:
# Define some initial guesses for m and c
guesses = (1, 1)

And now we can call `curve_fit` to fit the data and plot the data along with the fit.

In [ ]:

# Get the fit
popt, pcov = curve_fit(func, xs, ys, p0=guesses)

# Compute the fit using the parameters in popt
fit = func(xs, popt[0], popt[1])

# Plot the resulting fit and data
plt.scatter(xs, ys, marker='+')
plt.plot(xs, fit, linestyle='--')

# Label axes
plt.xlabel(r'$x$')
plt.ylabel(r'$y$')

plt.show()


On the topic of the guesses, `curve_fit` is fairly capable of handling even bad guesses so anything in the ball park of the right values should suffice. If you provide guesses miles from the truth it is possible for the fitting to fail, try it and see what happens. 

Take a look at what is returned by `curve_fit`, `popt` contains the optimal parameters in the same order they appear in the arguments of the function, `pcov` on the other hand is the covariance matrix, this is a concept from statistics which you will come across in the future so we will skirt around the complexities but it is enough right now to say the diagonal elements of `pcov` are the uncertainties on the returned parameters.

Although not covered here its worth noting that `curve_fit` can also take errors on the data and fold these into the fit. In the above we didn't have any errors/uncertainties but note that this is possible.

Hopefully this serves as a brief introduction to using `curve_fit` and it won't cause you distress when you come across it in the future. We will cover another example in the exercises. 


## Integration

Another staple in scientific computing is the numerical computation of integrals, with some integrals not having analytical solutions at all. You may have come across ways to approximate integrals in the past such as the trapezium and Simpson's rules. There are multiple implementations of these from different modules, but here I'll briefly cover the `scipy.integrate` versions.  

To compute an integral with these functions we will define two arrays one of x values and one containing the result from a 1D Gaussian, normalised such that the integral should be unity. We can then apply the functions and see if we sampled with fine enough resolution to get the expected result.

Note that these functions can be used in a number of ways, either providing x values or an interval $dx$ at which to calculate the integration.

In [ ]:
from scipy.integrate import simps, trapz

# Define some xs
xs = np.linspace(-50, 50, 10000)

# Define a 1D Gaussian function
def gauss(x, mx=0, sigx=1):
    norm = 1. / np.sqrt(2. * np.pi * sigx**2)
    exponent = - (x - mx)**2. / (2. * sigx**2.)
    return norm * np.exp(exponent)

# Compute the Gaussian
ys = gauss(xs)

# Compute the integral by the trapezium and Simpson's rule 
trap = trapz(ys, xs)
simp = simps(ys, xs)

print(trap, simp)





Evidently in both cases we got (roughly) the expected result. This means we sampled with high enough resolution and over a large enough range. That said, for such a simple function this shouldn't be a surprise.

In addition to the trapezium and Simpson's rules `scipy` also contains a function called `quad` which allows you to compute an integral from a function. Let's reproduce the above example but using `quad`. To use `quad` we need to define a 1D Gaussian as a function and define upper ($b$) and lower ($a$) bounds for the integration.



In [ ]:
from scipy.integrate import quad

# Define some xs
xs = np.linspace(-5, 5, 10000)

# Define a 1D Gaussian function
def gauss(x, mx=0, sigx=1):
    norm = 1. / np.sqrt(2. * np.pi * sigx**2)
    exponent = - (x - mx)**2. / (2. * sigx**2.)
    return norm * np.exp(exponent)

# Compute the integral by the trapezium and simpsons rule 
qu = quad(gauss, a=-50, b=50)

print(qu)






At first glance `quad` didn't give as perfect a result as the simpler methods above, however one nice thing about `quad` is it also computes an error on the integration. Taking this error into account the result is indeed consistent with the expected result of unity. The inclusion of an error makes `quad` very useful and as functions get more complex `quad` really begins to shine. 

We have not covered the entirety of the possible integration methods provided by `scipy`, for a full list and a tutorial see the [docs](https://docs.scipy.org/doc/scipy/reference/tutorial/integrate.html).

### Exercises

- Generate the 1D Gaussian from the examples in the histogram section but this time taking only 1000 values.
- Create a histogram of the random samples.
- Define a Gaussian function and fit it. Plot the resulting histogram and fit as a bar plot and dashed line respectively. (Hint: you can make the normalisation a fitting parameter as well).
- Integrate $sin^2(x)cos^3(x)$ and $sin^2(x)cos^4(x)$ between $-2\pi$ and $2\pi$. Before you calculate them do you know immediately which should be 0? There's a hidden integration trick here with multiplying odd and even functions when combined with symmetric limits.



### Interpolation

Interpolation is likely to come up less in your degree than the previous two operations but is a very useful tool to have in the tool box regardless. Interpolation is the process of taking a set of x and y data (or indeed multidimensional data) and finding the value of y at a value of x not included in the original dataset. This is why it is called *inter*polation, it only uses information inside, unlike *extra*polation, which gives values outside of the original range (and is very bad).

Interpolation can be used to get the value of a function at an unmeasured point or increase the resolution of your data (but this should always be done with caution). Below I'll demonstrate increasing the resolution of a simple sin curve with `scipy`'s 1D dimensional interpolation function (`interp1d`). 

First off we will need the original low resolution data.

In [ ]:
# Define a sin wave
xs = np.linspace(-2 * np.pi, 2 * np.pi, 15)
ys = np.sin(xs)

We can plot this and see we have a very badly sampled sin wave.

In [ ]:
plt.plot(xs, ys)
plt.show()

Now we can interpolate it. To do so we need a to use `interp1d` to create an interpolated function that can be evaluated at a new set of x values. In fact, we'll make two for reasons that I'll describe below.

In [ ]:
from scipy.interpolate import interp1d


# Create the interpolation function
lin_func = interp1d(xs, ys, kind="linear")
cub_func = interp1d(xs, ys, kind="cubic")

You'll see the `"kind"` argument above for the two functions. There are multiple different kinds of interpolation, the differences being how the values between data points are derived. With "linear" interpolation a straight line is drawn between adjacent points and values inbetween data points are sampled from this line. For "cubic" interpolation a cubic polynomial is used. For a full list of possible types see the [docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp1d.html).

Now lets sample the interpolated function and plot the result.

In [ ]:
# Define some new higher resolution x values
new_xs = np.linspace(-2 * np.pi, 2 * np.pi, 1000)

# Evaluated the interpolated function
new_lin_ys = lin_func(new_xs)
new_cub_ys = cub_func(new_xs)

plt.plot(xs, ys, linestyle="dotted", label="original")
plt.plot(new_xs, new_lin_ys, label="linear", linestyle="--")
plt.plot(new_xs, new_cub_ys, label="cubic", linestyle="-")
plt.plot(new_xs, np.sin(new_xs), linestyle="dotted", label="truth")
plt.legend(loc="upper right")
plt.show()

You can see from the above that the linear version followed the original low resolution data perfectly but the cubic gave a much better approximation of the full sin wave. 

I won't cover it explictly here but `scipy` has functions that will interpolate images (`interp2d`) or arbitrary dimensional datasets (`make_interp_spline`) but with that (and very little fanfare) you've reached the end of the notebooks!